In [2]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader


path = "/Users/admin/Working/thaibinh-chatbot/input"
loader = DirectoryLoader(path, glob="**/[!.]*")
docs = loader.load()
docs

/opt/anaconda3/envs/test_ragas3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '/Users/admin/Working/thaibinh-chatbot/input/Quy chế Học sinh 3 rèn luyện.pdf'}, page_content='BAN CHẤP HÀNH TRUNG ƯƠNG *** Số: - QĐ/TWĐTN-TNTH\n\nĐOÀN TNCS HỒ CHÍ MINH\n\nHà Nội, ngày tháng năm 2024\n\nQUYẾT ĐỊNH Về việc ban hành Quy chế danh hiệu “Học sinh 3 rèn luyện” ----------\n\nBAN BÍ THƯ TRUNG ƯƠNG ĐOÀN TNCS HỒ CHÍ MINH\n\nCăn cứ Điều 31, Chương IX, Điều lệ Đoàn TNCS Hồ Chí Minh; - Căn cứ Quy chế hoạt động của BCH Trung ương Đoàn (khoá XII); Quy\n\nchế tổ chức và hoạt động của cơ quan Trung ương Đoàn;\n\nXét đề nghị của Ban Thanh niên Trường học Trung ương Đoàn,\n\nQUYẾT ĐỊNH\n\nĐiều 1. Ban hành kèm theo quyết định này Quy chế danh hiệu “Học sinh 3 rèn luyện” dành cho học sinh đang đào tạo trình độ trung cấp trong các cơ sở giáo dục nghề nghiệp.\n\nĐiều 2. Ban Thanh niên Trường học Trung ương Đoàn là cơ quan thường trực danh hiệu, có trách nhiệm tham mưu Ban Bí thư Trung ương Đoàn chỉ đạo tổ chức thực hiện quy định và trao tặng danh hiệu “Học sinh 

In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from llm.get_llm import generate_test_model_function, get_embedding_function, get_model_function
generator_llm = LangchainLLMWrapper(get_model_function())
generator_embeddings = LangchainEmbeddingsWrapper(get_embedding_function())

✅✅Environment file in llm loaded successfully
get llm 
get_embedding_function


In [18]:
from ragas.testset.persona import Persona

personas = [
    Persona(
        name="sinh viên tò mò",
        role_description="Một sinh viên Việt Nam đang tiến hành nghiên cứu về các chủ đề liên quan đến hoạt động hội sinh viên và các sự kiện của đoàn trường. Sinh viên này sẽ đặt tất cả các câu hỏi bằng tiếng Việt.",
    ),
]

In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings
)

In [20]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
]

# for query, _ in distribution:
    # prompts = await query.adapt_prompts("spanish", llm=generator_llm)
    # query.set_prompts(**prompts)
    # query.save_prompts("/Users/admin/Working/thaibinh-chatbot/evaluate")

In [21]:
dataset = generator.generate_with_langchain_docs(
    docs,
    testset_size=30,
    query_distribution=distribution
)

Generating Samples: 100%|██████████| 30/30 [00:09<00:00,  3.15it/s]


In [23]:
eval_dataset = dataset.to_evaluation_dataset()
eval_dataset.to_csv("testset_5files2.csv")